In [ ]:
import streamlit as st
import os
import googleapiclient.discovery
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# Streamlit 페이지 설정
st.set_page_config(
    page_title="인기 유튜브 동영상",
    page_icon="🎬",
    layout="wide"
)

# YouTube API 키 가져오기
API_KEY = os.getenv("YOUTUBE_API_KEY")

@st.cache_data(ttl=3600)  # API 응답을 1시간 동안 캐싱
def get_popular_videos():
    """YouTube API를 호출하여 인기 동영상 리스트를 가져옵니다."""
    if not API_KEY:
        st.error("🚨 환경 변수 'YOUTUBE_API_KEY'를 설정하지 않았습니다.")
        return None

    try:
        # YouTube API 클라이언트 생성
        youtube = googleapiclient.discovery.build(
            "youtube", "v3", developerKey=API_KEY
        )

        # 인기 동영상 리스트 요청
        request = youtube.videos().list(
            part="snippet,statistics",
            chart="mostPopular",
            regionCode="KR",  # 한국(KR) 지역 기준
            maxResults=30     # 최대 30개 결과
        )
        response = request.execute()

        return response.get("items", [])

    except Exception as e:
        st.error(f"😭 API 호출 중 오류가 발생했습니다: {e}")
        return None

# --- UI 구성 ---
st.title("🔥 인기 유튜브 동영상")

# 새로고침 버튼
if st.button("새로고침"):
    # 캐시를 비우고 다시 API 호출
    st.cache_data.clear()
    st.experimental_rerun()

# 동영상 리스트 표시
video_items = get_popular_videos()

if video_items:
    for video in video_items:
        try:
            # 동영상 정보 추출
            snippet = video["snippet"]
            statistics = video["statistics"]
            
            title = snippet["title"]
            channel_title = snippet["channelTitle"]
            thumbnail_url = snippet["thumbnails"]["high"]["url"]
            view_count = int(statistics.get("viewCount", 0))
            
            # 동영상 컨테이너 생성 (썸네일, 제목, 채널명, 조회수)
            col1, col2 = st.columns([1, 4])
            
            with col1:
                st.image(thumbnail_url, use_column_width="auto")
                
            with col2:
                st.subheader(title)
                st.write(f"**채널:** {channel_title}")
                # 조회수 포맷팅
                st.write(f"**조회수:** {view_count:,}회")
            
            st.markdown("---")  # 구분선
            
        except KeyError as e:
            st.warning(f"⚠️ 일부 동영상 데이터가 누락되었습니다: {e}")
            continue